In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import tools, algorithms, collection, ui

In [3]:
from ipygee import *

In [4]:
Map = Map()
Map.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## BRDF correction in Landsat 8 TOA

In [4]:
i = ee.Image('LANDSAT/LC8_L1T_TOA_FMASK/LC81780592016030LGN00').select(
    ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B9', 'B10', 'fmask'],
    ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'cirrus', 'thermal', 'fmask']
);

In [5]:
ic = algorithms.Landsat.brdfCorrect(i)

In [6]:
p = ee.Geometry.Point([22.0227, 1.4801])
Map.centerObject(p, zoom=10)

In [7]:
ival = 0.10098563879728317
icval = 0.10012929931321499

In [8]:
ival_here = tools.image.getValue(i, p, 30, 'client')['blue']

In [9]:
ival_here == ival

True

In [10]:
icval_here = tools.image.getValue(ic, p, 30, 'client')['blue']

In [11]:
icval_here == icval

True

In [12]:
vis = {'bands':['nir', 'swir1','red'], 'min':0, 'max': 0.5}

In [13]:
Map.addLayer(i, vis, 'original')

In [14]:
Map.addLayer(ic, vis, 'corrected')

## Using `collection` module

In [15]:
l8toa = collection.Landsat8TOA()

In [16]:
vis = l8toa.visualization('NSR')

In [17]:
p = ee.Geometry.Point([22.21, 1.5])

In [18]:
col = l8toa.collection.filterBounds(p)

In [21]:
i2 = col.first()

In [28]:
proxy = ee.Image.constant(0).rename('proxy')

In [29]:
colr = col.map(lambda img: l8toa.rename(img)).map(lambda img: img.addBands(proxy))

In [30]:
colbrdf = colr.map(lambda img: l8toa.brdf(img, True))

In [22]:
Map.addLayer(i2, vis, 'Landsat 8 TOA')

In [31]:
i2c = colbrdf.first()

In [32]:
Map.addLayer(i2c, l8toa.visualization('NSR', True), 'Landsat 8 TOA BRDF Corrected')

In [25]:
diff = i2.subtract(i2c).multiply(100)

In [26]:
Map.addLayer(diff, vis.update(dict(min=-1, max=1)), 'BRDF difference')